In [1]:
import pandas as pd
import random
import sys
import numpy as np

sys.path.append("../scripts")
from scripts.prompt_gen import generate_prompt
from scripts.translate_summary_json import translate_summary

%load_ext autoreload
%autoreload 2

In [2]:
df = pd.read_csv("../outputs/cleaned_augmented_clinical_notes.csv")
q_df = pd.read_csv("../outputs/Questionnaire - Questions.csv")

In [24]:
def generate_record_dict ():
    record_dict = {
        "idx" : 0,
        "full_note" : "",
        "summary" : "",
        "questions" : "",
        "section_1" : "",
        "section_2" : "",
        "section_1_ground_truth_summary" : "",
        "section_2_ground_truth_summary" : "",
        "ground_truth_summary" : "",
        # "gpt3.5_preds" : "",
        # "gpt3.5_BERT_SCORER_agg" : "",
        # "gpt3.5_BERT_SCORER_section_1" : "",
        # "gpt3.5_BERT_SCORER_section_2" : "",
        # "llama2_preds" : "",
        # "llama2_BERT_SCORER_agg_section_1" : "",
        # "llama2_BERT_SCORER_agg_section_2" : "",
        # "llama2_BERT_SCORER_agg" : "",
        "prompt" : "",
    }
    return record_dict
    

record_list = []
for i in range(100):
    prompt_details = generate_prompt(df, q_df, return_prompt_string= False)
    
   
    record_dict = generate_record_dict()
    record_dict["idx"] += prompt_details["idx"]
    record_dict["prompt"] = prompt_details["prompt"]
    record_dict["full_note"] = prompt_details["full_note"]
    record_dict["summary"] = prompt_details["summary"]
    record_dict["questions"] = prompt_details["questions"]
    record_dict["section_1"] = prompt_details["sections"][0]
    record_dict["section_2"] = prompt_details["sections"][1]

    record_list.append(record_dict)

In [25]:
for record_dict in record_list:
    translated_summary, summary_dict = translate_summary(record_dict["summary"])
    record_dict["ground_truth_summary"] = translated_summary
    record_dict["section_1_ground_truth_summary"] = "".join(summary_dict[record_dict["section_1"]])
    record_dict["section_2_ground_truth_summary"] = "".join(summary_dict[record_dict["section_2"]])


In [23]:
df_records = pd.DataFrame(record_list)

In [26]:
df_records.to_csv("../outputs/eval.csv", index=False)